# 1️. Model Architecture Overview

  **Multi-Scale 1-D CNN** zur Objekterkennung.

  - **Input**: 1‑D Signal der Länge 163.
  - **Parallel feature paths**: 2–4 convolutional Zweige mit verschiedenen Kernel-Größen und Anzahl an Filtern.
  - **Global pooling**: Erzeugt eine Darstellung, die unabhängig vom Startpunkt des Scans ist.
  - **Dense head**: Klassifiziert in verschiedene Objekte.

In [2]:

# Imports
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from sklearn.model_selection import KFold
from scipy.ndimage import median_filter
from scipy.signal import savgol_filter
from training_pc.src.utils.data_processing import SmartAugmentor
import numpy as np

# 2. Daten Preprocessing

Bevor die Daten zum Trainieren des Models genutzt werden können, muss zuerst mit Ausreißern umgegangen werden. Zudem können die Daten noch gelättet werden, um weniger Rauschen zu bekommen.

 - **Ausreißer**: durch eine festgelegte maximale Distanz, können Messfehler durch den Sensor behoben werden. **Ausreißer --> Nan --> Interpolieren**; bereits bestehende NaN werden vorher rausgeworfen
  - **Median-Filter**: entfernt noise
  - **Savitzky-Golay-Filter**: "glättet" die Daten

In [ ]:
class DataProcessor:
    def __init__(self, max_dist=15.0, steps=163):
        self.max_dist = max_dist
        self.steps = steps

    def clean_scan(self, raw_distances):
        denoised = median_filter(raw_distances, size=3)
        smoothed = savgol_filter(denoised, window_length=11, polyorder=2)
        return smoothed

    def process_file(self, file_path, label_idx):
        df = pd.read_csv(file_path, comment="#").dropna(subset=["distance_cm"])

        df.loc[df["distance_cm"] > self.max_dist, "distance_cm"] = np.nan

        df["distance_cm"] = df.groupby("scan_index")["distance_cm"].transform(
            lambda x: x.interpolate().fillna(self.max_dist)
        )

        scans, labels = [], []

        for s_id in df["scan_index"].unique():
            scan = df[df["scan_index"] == s_id]["distance_cm"].values
            cleaned = self.clean_scan(scan)
            padded = np.pad(cleaned, (0, max(0, self.steps - len(cleaned))), mode='edge')[:self.steps]

            scans.append(padded / self.max_dist)
            labels.append(label_idx)

        return np.array(scans), np.array(labels)

# 3. Augmentation

  Durch das Augmentieren der Daten lernt das Model auch leicht veränderte Messungen durch zufällige Faktoren beim Messen zu erkennen. Außerdem wird das Model durch die leicht veränderten Daten robuster gegen Overfitting.

 - ``rotate_scan``: die Daten werden "rotiert", sodass, jeder Scan einen anderen Startpunkt hat
 - ``add_noise``: jedem Datenpunkt wird ein minimaler noise-Wert hinzugefügt (0 - 0.02)
 - ``jitter_scale``: simuliert verschiebung des Objektes auf dem Drehteller.


In [ ]:
class SmartAugmentor:
    @staticmethod
    def rotate_scan(scan, shift_range=20):
        shift = np.random.randint(-shift_range, shift_range)
        return np.roll(scan, shift, axis=0)

    @staticmethod
    def add_noise(scan, noise_level=0.02):
        return scan + np.random.normal(0, noise_level, scan.shape)

    @staticmethod
    def jitter_scale(scan, factor=0.05):
        return scan * (1 + np.random.uniform(-factor, factor))

# 4. Das Modell

Wir verwenden ein Multi-Scale 1D-Convolutional Neural Network. Mit dem Multi-Scale Ansatz erreichen wir, dass das Modell einerseits kleine Feinheiten als auch die grobe Form erkennen kann. Das Convolutional Layer selbst ist da, um Formen zu erkennen.

In [ ]:
def build_model(hp):
    """Factory function for the Multi-Scale 1D-CNN architecture."""
    inputs = keras.Input(shape=(163, 1))
    path_outputs = []

    for i in range(hp.Int('num_paths', 2, 4)):
        kernel_size = hp.Choice(f'kernel_{i}', [3, 7, 11, 15])
        filters = hp.Int(f'filters_{i}', 16, 64, step=16)

        x = layers.Conv1D(filters, kernel_size, padding='same', activation='relu')(inputs)
        x = layers.BatchNormalization()(x)
        path_outputs.append(x)
    merged = layers.Concatenate()(path_outputs)
    x = layers.GlobalMaxPooling1D()(merged)

    x = layers.Dense(hp.Int('dense_units', 32, 128, step=32), activation='relu')(x)
    x = layers.Dropout(hp.Float('dropout', 0.1, 0.4, step=0.1))(x)

    outputs = layers.Dense(5, activation='softmax')(x)

    model = keras.Model(inputs, outputs)

    lr = hp.Float('lr', 1e-4, 1e-2, sampling='log')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model